<a href="https://colab.research.google.com/github/ImaduddinAhmedMohammed/ImaduddinAhmed_INFO5731_Spring2024/blob/main/LLM_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install nltk

In [17]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

data=pd.read_csv("modelling_LLM.csv")

# Define a function to preprocess the text
def preprocess_text(text):
    # Check if the input is a string
    if not isinstance(text, str):
        return ''  # Return an empty string or a default value for non-string inputs

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]



    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text
data.dropna(subset=['comment_depth', 'comment_body', 'comment_likes', 'start', 'Label'], inplace=True)
data = data[['comment_depth', 'comment_body', 'comment_likes', 'start', 'Label']]

# Apply the preprocessing function to the comment_body column
data['Cleaned_comment_body'] = data['comment_body'].apply(preprocess_text)

data.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,comment_depth,comment_body,comment_likes,start,Label,Cleaned_comment_body
0,0.0,It's way better at powershell than I am for su...,356.0,1,1,way better powershell sure occasionally calls ...
1,1.0,[deleted],67.0,0,0,deleted
2,2.0,Exactly this. I'm...average at powershell. G...,37.0,0,2,exactly average powershell gpt average
3,3.0,Don't tell that to GPT or else it might demand...,20.0,0,3,tell gpt else might demand raise want feel emo...
4,4.0,If you say youâ€™re going to give it a tip it ...,8.0,0,3,say going give tip better job also quality inc...


In [18]:
data.to_csv("cleaned_modelling_LLM.csv", index=False)

In [19]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [20]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Load data from CSV file
data = pd.read_csv("cleaned_modelling_LLM.csv")

# Drop any rows with missing values in 'Cleaned_comment_body' if necessary
data.dropna(subset=['Cleaned_comment_body'], inplace=True)

# Load tokenizer and model from the transformers library
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.eval()  # Set the model to eval mode to disable training specific behaviors

# Function to tokenize and extract features
def process_data(data, batch_size=1200):  # Smaller batch size to manage memory usage
    features = []
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)

    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        tokens = tokenizer(batch['Cleaned_comment_body'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
        with torch.no_grad():
            outputs = model(input_ids=tokens['input_ids'], attention_mask=tokens['attention_mask'])
        features.append(outputs.last_hidden_state[:, 0, :])  # Extracting the [CLS] token features
        print(f"Processed batch {i // batch_size + 1}/{total_batches}")
    print("All data processed.")
    return torch.cat(features)

# Assuming you have a way to define or split train_data and test_data
# Since it's not defined how to split the data, here's a simple approach:
from sklearn.model_selection import train_test_split

# Split data into training and testing
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Process data
train_features = process_data(train_data)
test_features = process_data(test_data)

# Now proceed with your further steps, like saving features or using them for evaluation


Processed batch 1/12
Processed batch 2/12
Processed batch 3/12
Processed batch 4/12
Processed batch 5/12
Processed batch 6/12
Processed batch 7/12
Processed batch 8/12
Processed batch 9/12
Processed batch 10/12
Processed batch 11/12
Processed batch 12/12
All data processed.
Processed batch 1/3
Processed batch 2/3
Processed batch 3/3
All data processed.


In [21]:
train_features

tensor([[-0.2272, -0.3327, -0.1031,  ..., -0.2145,  0.1684,  0.1891],
        [ 0.2930,  0.0025, -0.0234,  ..., -0.3053,  0.4014,  0.2601],
        [-0.1391, -0.0626, -0.0546,  ..., -0.5970,  0.4655, -0.1534],
        ...,
        [-0.3052, -0.1074,  0.1374,  ..., -0.1726,  0.3329,  0.4307],
        [-0.3370, -0.1268, -0.0432,  ..., -0.0970,  0.0498,  0.3817],
        [-0.4469, -0.0766,  0.0845,  ..., -0.4433,  0.5401,  0.3247]])

In [22]:
!pip install transformers[torch]

In [23]:
import accelerate
import transformers
from transformers import Trainer, TrainingArguments
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

Accelerate version: 0.21.0
Transformers version: 4.40.1


In [24]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import GPT2Tokenizer, GPT2Model, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load tokenizer and base GPT-2 model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
base_model = GPT2Model.from_pretrained(model_name)

# Extend GPT-2 with a sequence classification head
class GPT2ForSequenceClassification(nn.Module):
    def __init__(self, base_model, num_labels):
        super(GPT2ForSequenceClassification, self).__init__()
        self.gpt2 = base_model
        self.config = base_model.config  # Make the config attribute available
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.gpt2.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        # Using the last token's output (conventional for tasks like classification)
        pooled_output = outputs.last_hidden_state[:, -1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Load and preprocess data
data = pd.read_csv("cleaned_modelling_LLM.csv")
data.dropna(subset=['Cleaned_comment_body'], inplace=True)
label_encoder = LabelEncoder()
data['labels'] = label_encoder.fit_transform(data['Label'])

# Encode data using the tokenizer
def encode_data(data, tokenizer):
    # Ensure encoding is performed correctly
    tokens = tokenizer(data['Cleaned_comment_body'].tolist(), max_length=128, padding="max_length", truncation=True, return_tensors="pt")

    # Create label tensor, ensuring it has the correct shape
    labels_tensor = torch.tensor(data['labels'].values)

    # Creating the dataset
    dataset = TensorDataset(tokens['input_ids'], tokens['attention_mask'], labels_tensor)

    # Splitting the dataset
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    return train_dataset, val_dataset

train_dataset, val_dataset = encode_data(data, tokenizer)



# Initialize the Trainer
#model = GPT2ForSequenceClassification(base_model, num_labels=2)
"""

# Train and evaluate the model
trainer.train()
results = trainer.evaluate()
print(results)
"""

'\n\n# Train and evaluate the model\ntrainer.train()\nresults = trainer.evaluate()\nprint(results)\n'

In [25]:
!pip install transformers

In [36]:
import accelerate
import transformers
from transformers import Trainer, TrainingArguments
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

Accelerate version: 0.21.0
Transformers version: 4.40.1


In [41]:
!pip install accelerate==0.30.0

In [47]:
!pip install transformers[torch]

In [48]:
import accelerate
import transformers
from transformers import Trainer, TrainingArguments
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

Accelerate version: 0.21.0
Transformers version: 4.40.1


In [45]:
from transformers import GPT2Model

base_model = GPT2Model.from_pretrained("gpt2")
model = GPT2ForSequenceClassification(base_model, num_labels=5)  # Adjust num_labels as needed

In [ ]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,                   # your model
    args=training_args,            # training arguments
    train_dataset=train_dataset,   # training dataset
    eval_dataset=val_dataset       # evaluation dataset
)


In [ ]:
print(model.config)

In [ ]:
config_dict = model.config.to_dict()
print(config_dict)

In [ ]:
print(type(model))  # Should show a model class from transformers.
print(type(train_dataset))  # Should show 'datasets.arrow_dataset.Dataset' or similar.
print(type(val_dataset))

In [ ]:
print(trainer.args)

In [ ]:
!pip install --upgrade transformers datasets

In [ ]:
# Check the length of the dataset
print("Dataset size:", len(val_dataset))

# If possible, print the first element to check its structure
if len(val_dataset) > 0:
    print("First element of the dataset:", val_dataset[0])

# If slicing might be an issue, try accessing a single element
if len(val_dataset) > 5:
    print("Sixth element of the dataset:", val_dataset[5])
else:
    print("Dataset has fewer than 6 elements, adjust the slice.")

In [ ]:
!pip install scikit-learn

In [ ]:
# Verify the dataset format
print("Type of dataset:", type(val_dataset))
print("First element of dataset:", val_dataset[0] if len(val_dataset) > 0 else "Dataset is empty")

# Verify the trainer's configuration
print("Trainer type:", type(trainer))


In [ ]:
# Trying a simple predict call with basic error handling
try:
    preds_output = trainer.predict(val_dataset)
    print("Prediction successful:", preds_output)
except TypeError as e:
    print("TypeError caught:", str(e))
    # Additional check to see what might be causing the TypeError
    print("Direct attribute check via vars() on trainer:", vars(trainer))


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, cohen_kappa_score
import numpy as np

# Use the Trainer to predict the outputs on the validation set
preds_output = trainer.predict(val_dataset)

# Debugging: Print the type and content of preds_output
print("Type of preds_output:", type(preds_output))
print("Content of preds_output:", preds_output)

# Ensure preds_output has the necessary attributes or keys
if 'predictions' in preds_output and 'label_ids' in preds_output:
    predictions = np.argmax(preds_output['predictions'], axis=1)
    true_labels = preds_output['label_ids']

    # Calculate the required metrics using scikit-learn
    accuracy = accuracy_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    cohen_kappa = cohen_kappa_score(true_labels, predictions)

    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"Cohen's Kappa: {cohen_kappa}")
else:
    raise AttributeError("The prediction output does not have the expected keys 'predictions' or 'label_ids'")


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, cohen_kappa_score
import numpy as np

# Use the Trainer to predict the outputs on the validation set
preds_output = trainer.predict(val_dataset)

# Debugging: Check the type and attributes of preds_output
print("Type of preds_output:", type(preds_output))
print("Attributes of preds_output:", dir(preds_output))

# Ensure preds_output has the necessary attributes
if hasattr(preds_output, 'predictions') and hasattr(preds_output, 'label_ids'):
    predictions = np.argmax(preds_output.predictions, axis=1)
    true_labels = preds_output.label_ids

    # Calculate the required metrics using scikit-learn
    accuracy = accuracy_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    cohen_kappa = cohen_kappa_score(true_labels, predictions)

    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"Cohen's Kappa: {cohen_kappa}")
else:
    raise AttributeError("The prediction output does not have the expected attributes")
